# How to do analysis on individual spectral lines

Many of the tools in SpecpolFlow can be applied to a single line in an observed spectrum.  In this example we analyze a few emission lines in the spectrum of a T Tauri star.

:::{Warning}
Still under construction
:::

First, import SpecpolFlow and some packages that will help with data visualization.

In [ ]:
import specpolFlow as pol

import pandas as pd
import matplotlib.pyplot as plt

## 1 . Create individual line file

In this tutorial, we use the UPENA normalized spectrum `IndividualLine_tutorialfiles/1423137pn.s` of the active cool star BP Tau.

We first load in the spectrum. 

In [ ]:
spec = pol.read_spectrum("IndividualLine_tutorialfiles/1423137pn.s")

To create the individual line file, we have to define the transition wavelength `lambda0` and its range with respect to the line center. In the example below, we select the HeI line at 667.815 nm in a 0.6 nm window.

In [ ]:
# Create individual line profile 
lambda0 = 667.815;  geff = 1.0 # He i 667.815 nm
lwidth = 0.3 

prof = spec.individual_line(lambda0 = lambda0, lwidth = lwidth)

The object `prof` inherits all the atributes from the LSD class. Therefore, you can use all the functionality discussed in the LSD Class Tutorial (i.e., one can normalize, slice, or shift the emission line profile).

Below, we visualize the individual line using the `plot` class function. 

In [ ]:
fig, axes = prof.plot()
axes[-1].axhline(1, color ='k', ls = '--')

## 2. Calculate Bz

Below, we demonstrate how to compute the longitudinal field (Bz) of the individual line. 

In [ ]:
# Define the limits for the continuum normalization
cmin = -50.0; cmax = 80.0

# Compute Bz in a 45 km/s window
Bz, fig = prof.calc_bz(norm = 'auto', cog = 'I', 
                   velrange = [cmin, cmax], bzwidth = 45.0, 
                   geff = geff, lambda0 = lambda0,
                   plot = True)

We can then display the results using the pandas dataframa. For this particular line, we find Bz $= 1.2 \pm 0.1$kG.

In [ ]:
Bz = pd.DataFrame(data = [Bz])

# Simple display of the pandas dataframe
Bz.style

## 3. Normalize the individual line

In the figure above, we can promptly note that `calc_bz` improved the continuum normalization, taking the median of the continuum outside of the line. 
Therefore, we propagate this normalization in the `prof` object before saving the individual line profile into a file.

In [ ]:
# Renormalize prof using the continuum value obtained by calc_bz
prof_norm = prof.norm(Bz['Ic'][0])

# Display the profile
fig, axes = prof_norm.plot()
axes[-1].axhline(1, color = 'k', ls = '--')

## 4. Saving individual lines

Finally, you can save the individual line into a file running the code below.

In [ ]:
prof_norm.save("IndividualLine_tutorialfiles/1423137.Helium")

## 5. Dealing with order overlaps

Now, let's do the same for the Halpha line:

In [ ]:
# reading the spectrum
spec = pol.read_spectrum("IndividualLine_tutorialfiles/1423137pn.s")
# defining the line parameters
lambda0 = 656.281 ;  geff = 1.0 # Halpha
lwidth = 700. * lambda0 / 2.99e5 #width in nm from width in km/s
# getting the profile for that line
prof = spec.individual_line(lambda0 = lambda0, lwidth = lwidth)

# Define the limits for the continuum normalization
# Compute Bz in a 300 km/s window around the cog
cogRange = [-400., 400.]
intRange = 200.
Bz, fig = prof.calc_bz(norm = 'auto', cog = 'I', 
                       velrange = cogRange, bzwidth = intRange, 
                       geff = geff, lambda0 = lambda0,
                       plot = True)   

# Creating a dataframe to record the result of the different methods in this section
d = {'Merge type':'None'}
d.update(Bz)
df_results = pd.DataFrame(data = [d])
df_results.style

### 5.1 Option 1: Selecting only one order

In [ ]:
## Selecting only one order
orders = spec.get_orders_in_range(lambda0)
print('There are {} orders with Halpha in them'.format(len(orders)))

# Illustration of the order overlap
fig, ax = plt.subplots(1,1)
ax.plot(orders[0].wl, orders[0].specI, label = 'Lower order')
ax.plot(orders[1].wl, orders[1].specI, label = 'Higher order')
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Stokes I / Ic')
ax.legend(loc = 0)

# Making a loop over the 
for ord, type in zip(orders,['lower only', 'higher only']):

# getting the profile for that line
    prof = ord.individual_line(lambda0 = lambda0, lwidth = lwidth)

    # Compute Bz, in the defined velocity window around the cog
    Bz, fig = prof.calc_bz(norm = 'auto', cog = 'I', 
                           velrange = cogRange, bzwidth = intRange, 
                           geff = geff, lambda0 = lambda0,
                           plot = True)
    d = {'Merge type':type}
    d.update(Bz)
    df = pd.DataFrame(data=[d])
    df_results = pd.concat([df_results,df], ignore_index=True)

# Display the Bz result as a dataframe
df_results.style

### 5.2 Option 2: merging the orders

In [ ]:
## Merging using the trim method
merge = spec.merge_orders(mode = 'trim', midpoint = 0.5)

prof = merge.individual_line(lambda0 = lambda0, lwidth = lwidth)
Bz, fig = prof.calc_bz(norm = 'auto', cog = 'I', 
                       velrange = cogRange, bzwidth = intRange, 
                       geff = geff, lambda0 = lambda0,
                       plot = True)
d = {'Merge type':'trim at mid-point'}
d.update(Bz)
df = pd.DataFrame(data = [d])
df_results = pd.concat([df_results,df], ignore_index=True)


## Merging using the coadd method
merge = spec.merge_orders(mode='coadd')

prof = merge.individual_line(lambda0 = lambda0, lwidth = lwidth)
Bz, fig = prof.calc_bz(norm = 'auto', cog = 'I', 
                       velrange = cogRange, bzwidth = intRange, 
                       geff = geff, lambda0 = lambda0,
                       plot = True)
d = {'Merge type':'coadd'}
d.update(Bz)
df = pd.DataFrame(data = [d])
df_results = pd.concat([df_results,df], ignore_index = True)

# Display the Bz result as a dataframe
df_results.style

The Halpha line is somewhat complicated to analyze, in that it has a very broad emission component from regions not associated with the magnetic field.  This makes the correct velocity range to use for integration in the Bz calculation ambiguous.

For a somewhat cleaner example that is still in a region where two spectral orders overlap, we can look at the He I 587.56 nm line.

In [ ]:
# reading the spectrum
spec = pol.read_spectrum("IndividualLine_tutorialfiles/1423137pn.s")
# defining the line parameters
lambda0 = 587.56 ;  geff = 1.0 # He I, really a multiplet (so geff is rather approximate)
lwidth = 500. * lambda0 / 2.99e5 #width in nm from width in km/s
# getting the profile for that line
prof = spec.individual_line(lambda0 = lambda0, lwidth = lwidth)

# Define the limits for the continuum normalization
# Compute Bz in a 300 km/s window around the cog
cogRange = [-100., 150.]
intRange = 50.
Bz, fig = prof.calc_bz(norm = 'auto', cog = 'I', 
                       velrange = cogRange, bzwidth = intRange, 
                       geff = geff, lambda0 = lambda0,
                       plot = True)   

# Creating a dataframe to record the result of the different methods in this section
d = {'Merge type':'None'}
d.update(Bz)
df_results = pd.DataFrame(data = [d])

orders = spec.get_orders_in_range(lambda0)
# Making a loop over the individual orders
for ord, type in zip(orders,['lower only', 'higher only']):
    # getting the profile for that line
    prof = ord.individual_line(lambda0 = lambda0, lwidth = lwidth)
    Bz = prof.calc_bz(norm = 'auto', cog = 'I', 
                           velrange = cogRange, bzwidth = intRange, 
                           geff = geff, lambda0 = lambda0,
                           plot = False)
    d = {'Merge type':type}
    d.update(Bz)
    df = pd.DataFrame(data = [d])
    df_results = pd.concat([df_results,df], ignore_index=True)

# Merging using the trim method
merge = spec.merge_orders(mode = 'trim', midpoint = 0.5)
prof = merge.individual_line(lambda0 = lambda0, lwidth = lwidth)
Bz = prof.calc_bz(norm = 'auto', cog = 'I', 
                  velrange = cogRange, bzwidth = intRange, 
                  geff = geff, lambda0 = lambda0,
                  plot = False)
d = {'Merge type':'trim at mid-point'}
d.update(Bz)
df = pd.DataFrame(data = [d])
df_results = pd.concat([df_results,df], ignore_index=True)

## Merging using the coadd method
merge = spec.merge_orders(mode = 'coadd')
prof = merge.individual_line(lambda0 = lambda0, lwidth = lwidth)
Bz = prof.calc_bz(norm = 'auto', cog = 'I', 
                  velrange = cogRange, bzwidth = intRange, 
                  geff = geff, lambda0 = lambda0,
                  plot = False)
d = {'Merge type':'coadd'}
d.update(Bz)
df = pd.DataFrame(data = [d])
df_results = pd.concat([df_results,df], ignore_index=True)

# Display the Bz result as a dataframe
df_results.style